In [ ]:
import numpy as np
import math
import itertools
import picos as pic

In [ ]:
def K_fun(m,k,n):

    Fun=0

    for q in range(0,m+1):
        Fun+= math.comb(n-k,m-q)*math.comb(k,q)*(-1)**q
    return Fun 

def comb_g(n,k):

    if n>=0 and k>=0:
        value=math.comb(n,k)
    else:
        value=0

    return value

def Beta(i,j,k,t,n):

    beta=0

    for u in range(n+1):

        beta+= comb_g(u,t)*comb_g(n-2*k,u-k)*comb_g(n-k-u,i-u)*comb_g(n-k-u,j-u)*(-1)**(u-t)

    return beta

In [20]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    #sdp.options["abs_*_tol"] = 1e-06
    
    sdp.set_objective('find') 
    
                 
    ##### Adding matrix constraints #####
    
    sdp.add_constraint(Ap[0]==K**2)
    sdp.add_constraint(Ap[n]==K)
    
    ##### NPA #####
        
    ### Constructing Beta matrices ###
    
    cl=1e-6*0
    
    cl2=1e-6*0
    
    cl3=1e-6*0
    
    BMatrix=[]
    
    for k in range(0,int(n/2)+1):
        
        Matrix=[[0]*(n-2*k+1) for _ in range(n-2*k+1)]

        for i in range(k,n-k+1):
            for j in range(k,n-k+1):
                for t in range(n+1):
                    
                    l=i+j-2*t
                    
                    if l>n or t>i or t>j:
                        
                        Matrix[i-k][j-k]+=0
                    
                    else:
                        
                        Matrix[i-k][j-k]+=(1/math.comb(n,l))*Ap[l]*Beta(i,j,k,t,n)
                
        BMatrix.append(pic.block(Matrix))

        Zero=pic.block(np.identity(n-2*k+1))

        sdp.add_constraint(BMatrix[k] >> -cl*Zero) #### B_j(x)>>0 ####
        
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 
    
    #sdp.add_list_of_constraints(((K*Ap[n-i]-Ap[i]) >= -cl3) for i in range(d)) 
    #sdp.add_list_of_constraints(((K*Ap[n-i]-Ap[i]) <= cl3) for i in range(d)) 
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        sdp.add_constraint(sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*Ap[j] for j in range(m+1)) >= -cl2)

    ### K*B_j-A_j>=0 ###
    
    for m in range(n+1):   
                                    
        sdp.add_constraint(                              
        sum((-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1)) >= -cl2)
    
    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        sdp.add_constraint(sum(K_fun(n-j,i,n)*Ap[i] for i in range(n+1)) >= -cl2)
    
    sol = sdp.solve(primals=False,solver='mosek')
    
    #sol = sdp.solve(primals=True)

    #print(np.array(BMatrix[0]))
    
    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        return False
    else:
        return sol.problemStatus 

D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],

[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],
[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],
[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

#Codes=[[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1]]
"""

#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")



#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d+1,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")
"""


#### The program prints True if the sdp is feasible and false if it is infeasible #### 

'\n\n#### Existing qubit codes ####\n\nfor code in Codes:\n    \n    n,K,d=code\n    \n    check=SDP(n,2**K,d,D)\n    \n    if check != True:\n        print("%s=%s" % (code,check))\n\nprint("Done")\n\n\n\n#### Not existing qubit codes (distance+1) ####\n\nfor code in Codes:\n    \n    n,K,d=code\n    \n    check=SDP(n,2**K,d+1,D)\n    \n    if check != False:\n        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))\n\nprint("Done")\n'

In [21]:
D=2

Codes=[[7,26,2],[9,112,2],[11,460,2],[13,1877,2],[15,7606,2]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 26, 2]=True
[9, 112, 2]=unknown
[11, 460, 2]=unknown
[13, 1877, 2]=unknown
[15, 7606, 2]=False


In [ ]:
D=2

Codes=[[7,3,3],[8,9,3],[9,13,3],[11,53,3],[12,89,3],[13,204,3],[14,324,3],[15,580,3]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

In [ ]:
D=2

Codes=[[7,1,4],[10,5,4],[11,7,4],[12,20,4],[13,40,4],[14,102,4],[15,150,4]]
    
for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

In [ ]:
D=2

Codes=[[13,3,5],[14,10,5],[15,18,5]]
    
for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

In [ ]:
Codes=[[14,3,5],[24,0,10]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],2**codes[1],codes[2],D)))